# **Amazon Lookout for Equipment** - Getting started
*Part 5 - Scheduling regular inference calls*

## Initialization
---
This repository is structured as follow:

```sh
. lookout-equipment-demo
|
├── data/
|   ├── interim                          # Temporary intermediate data
|   ├── processed                        # Finalized datasets
|   └── raw                              # Immutable original data
|
├── getting_started/
|   ├── 1_data_preparation.ipynb
|   ├── 2_dataset_creation.ipynb
|   ├── 3_model_training.ipynb
|   ├── 4_model_evaluation.ipynb
|   ├── 5_inference_scheduling.ipynb           <<< THIS NOTEBOOK <<<
|   ├── 6_visualization_with_quicksight.ipynb
|   └── 7_cleanup.ipynb
|
└── utils/
    └── lookout_equipment_utils.py
```

### Notebook configuration update

In [ ]:
!pip install --quiet --upgrade sagemaker lookoutequipment

### Imports

In [ ]:
import boto3
import config
import datetime
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import os
import pandas as pd
import pytz
import sagemaker
import sys
import time

from matplotlib.gridspec import GridSpec

# SDK / toolbox for managing Lookout for Equipment API calls:
import lookoutequipment as lookout

### AWS Look & Feel definition for Matplotlib

In [ ]:
from matplotlib import font_manager

# Load style sheet:
plt.style.use('../utils/aws_matplotlib_template.py')

# Get colors from custom AWS palette:
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

### Parameters

In [ ]:
TMP_DATA       = os.path.join('..', 'data', 'interim', 'getting-started')
PROCESSED_DATA = os.path.join('..', 'data', 'processed', 'getting-started')
INFERENCE_DATA = os.path.join(PROCESSED_DATA, 'inference-data')
TRAIN_DATA     = os.path.join(PROCESSED_DATA, 'training-data', 'centrifugal-pump')

os.makedirs(INFERENCE_DATA, exist_ok=True)
os.makedirs(os.path.join(INFERENCE_DATA, 'input'), exist_ok=True)
os.makedirs(os.path.join(INFERENCE_DATA, 'output'), exist_ok=True)

ROLE_ARN                 = sagemaker.get_execution_role()
REGION_NAME              = boto3.session.Session().region_name
BUCKET                   = config.BUCKET
PREFIX                   = config.PREFIX_INFERENCE
INFERENCE_SCHEDULER_NAME = config.INFERENCE_SCHEDULER_NAME
MODEL_NAME               = config.MODEL_NAME

%matplotlib inline

## Create an inference scheduler
---
While navigating to the model details part of the console, you will see that you have no inference scheduled yet:

![Schedule Starting point](assets/schedule_start.png)

### Scheduler configuration
Let's create a new inference schedule: some parameters are mandatory, while others offer some added flexibility.

#### Mandatory Parameters

* Set `upload_frequency` at which the data will be uploaded for inference. Allowed values are `PT5M`, `PT10M`, `PT15M`, `PT30M` and `PT1H`.
  * This is both the frequency of the inference scheduler and how often data are uploaded to the source bucket.
  *  **Note**: ***the upload frequency must be compatible with the sampling rate selected at training time.*** *For example, if a model was trained with a 30 minutes resampling, asking for 5 minutes won't work and you need to select either PT30M and PT1H for this parameter at inference time.*
* Set `input_bucket` to the S3 bucket of your inference data
* Set `input_prefix` to the S3 prefix of your inference data
* Set `output_bucket` to the S3 bucket where you want inference results
* Set `output_prefix` to the S3 prefix where you want inference results
* Set `role_arn` to the role to be used to **read** data to infer on and **write** inference output

#### Time zone parameter (optional)

You can set `INPUT_TIMEZONE_OFFSET` to the following allowed values: `+00:00`, `+00:30`, `+01:00`, ... `+11:30`, `+12:00`, `-00:00`, `-00:30`, `-01:00`, ... `-11:30`, `-12:00`.

This is the timezone the scheduler will use to find the input files to run inference for. A timezone's offset refers to how many hours the timezone is from Coordinated Universal Time (UTC).

Let's take an example:
* The current date April 5th, 2021 and time is 1pm UTC
* You're in India, which is 5 hour 30 ahead of UTC and you set the `INPUT_TIMEZONE_OFFSET` to `+05:30`
* If the scheduler wakes up at 1pm UTC, A filename called 20210405**1830**00 will be found (1pm + 5H30 = 6.30pm)

Use the following cell to convert time zone identifier (`Europe/Paris`, `US/Central`...) to a time zone offset. You can build a timezone object by leveraging the World Timezone Definition **[available here](https://gist.github.com/heyalexej/8bf688fd67d7199be4a1682b3eec7568)** or by listing the available ones using this code snippet:
```python
import pytz
for tz in pytz.all_timezones:
    print tz
```
If you want to use universal time, replace the timezone string below (`Asia/Calcutta`) by `UTC`:

In [ ]:
utc_timezone = pytz.timezone("UTC")

# current_timezone = pytz.timezone("Asia/Calcutta")
current_timezone = pytz.timezone("UTC")
tz_offset = datetime.datetime.now(current_timezone).strftime('%z')
tz_offset = tz_offset[:3] + ':' + tz_offset[3:]
tz_offset

#### Other optional parameters

* Set `delay_offset` to the number of minutes you expect the data to be delayed to upload. It's a time buffer to upload data.
* Set `timestamp_format`. The allowed values `EPOCH`, `yyyy-MM-dd-HH-mm-ss` or `yyyyMMddHHmmss`. This is the format of timestamp which is the suffix of the input data file name. This is used by Lookout Equipment to understand which files to run inference on (so that you don't need to remove previous files to let the scheduler finds which one to run on).
* Set `component_delimiter`. The allowed values `-`, `_` or ` `. This is the delimiter character that is used to separate the component from the timestamp in the input filename.

### Create the inference scheduler
The CreateInferenceScheduler API creates a scheduler. The following code prepares the configuration but does not create the scheduler just yet:

In [ ]:
scheduler = lookout.LookoutEquipmentScheduler(
    scheduler_name=INFERENCE_SCHEDULER_NAME,
    model_name=MODEL_NAME
)

scheduler_params = {
    'input_bucket': BUCKET,
    'input_prefix': f'{PREFIX}/input/',
    'output_bucket': BUCKET,
    'output_prefix': f'{PREFIX}/output/',
    'role_arn': ROLE_ARN,
    'upload_frequency': 'PT5M',
    'delay_offset': None,
    'timezone_offset': tz_offset,
    'component_delimiter': '_',
    'timestamp_format': 'yyyyMMddHHmmss'
}

scheduler.set_parameters(**scheduler_params)

## Prepare the inference data
---
Let's prepare and send some data in the S3 input location our scheduler will monitor: we are going to extract 10 sequences of 5 minutes each (5 minutes being the minimum scheduler frequency). We assume that data are sampled at a rate of one data point per minute meaning that each sequence will be a CSV with 5 rows (to match the scheduler frequency). We have set aside a file we can use for inference. We need to update the timestamps to match the current time and date and then split the file in individual datasets of 5 rows each.

In [ ]:
# Load the original inference data:
inference_fname = os.path.join(TMP_DATA, 'inference-data', 'inference.csv')
inference_df = pd.read_csv(inference_fname)
inference_df['Timestamp'] = pd.to_datetime(inference_df['Timestamp'])
inference_df = inference_df.set_index('Timestamp')

# How many sequences do we want to extract:
num_sequences = 12

# The scheduling frequency in minutes: this **MUST** match the
# resampling rate used to train the model:
frequency = 5
start = inference_df.index.min()
for i in range(num_sequences):
    end = start + datetime.timedelta(minutes=+frequency - 1)
    inference_input = inference_df.loc[start:end, :]
    start = start + datetime.timedelta(minutes=+frequency)
    
    # Rounding time to the previous X minutes 
    # where X is the selected frequency:
    filename_tm = datetime.datetime.now(current_timezone)
    filename_tm = filename_tm - datetime.timedelta(
        minutes=filename_tm.minute % frequency,
        seconds=filename_tm.second,
        microseconds=filename_tm.microsecond
    )
    filename_tm = filename_tm + datetime.timedelta(minutes=+frequency * (i))
    current_timestamp = (filename_tm).strftime(format='%Y%m%d%H%M%S')
    
    # The timestamp inside the file are in UTC and are not linked to the current timezone:
    timestamp_tm = datetime.datetime.now(utc_timezone)
    timestamp_tm = timestamp_tm - datetime.timedelta(
        minutes=timestamp_tm.minute % frequency,
        seconds=timestamp_tm.second,
        microseconds=timestamp_tm.microsecond
    )
    timestamp_tm = timestamp_tm + datetime.timedelta(minutes=+frequency * (i))
    
    # We need to reset the index to match the time 
    # at which the scheduler will run inference:
    new_index = pd.date_range(
        start=timestamp_tm,
        periods=inference_input.shape[0], 
        freq='1min'
    )
    inference_input.index = new_index
    inference_input.index.name = 'Timestamp'
    inference_input = inference_input.reset_index()
    inference_input['Timestamp'] = inference_input['Timestamp'].dt.strftime('%Y-%m-%dT%H:%M:%S.%f')
    
    # Export this file in CSV format:
    scheduled_fname = os.path.join(INFERENCE_DATA, 'input', f'centrifugal-pump_{current_timestamp}.csv')
    inference_input.to_csv(scheduled_fname, index=None)
    
# Upload the whole folder to S3, in the input location:
!aws s3 cp --recursive --quiet $INFERENCE_DATA/input s3://$BUCKET/$PREFIX/input

Our S3 bucket is now in the following state: this emulates what you could expect if your industrial information system was sending a new sample of data every five minutes.

Note how:
* Every files are located in the same folder
* Each file has the recorded timestamp in its name
* The timestamps are rounding to the closest 5 minutes (as our scheduler is configured to wake up every 5 minutes)

![Inference input](assets/schedule_inference_s3_data.png)

Now that we've prepared the data, we can create the scheduler by running:

```python
create_scheduler_response = lookout_client.create_inference_scheduler({
    'ClientToken': uuid.uuid4().hex
})
```

The following method encapsulates the call to the [**CreateInferenceScheduler**](https://docs.aws.amazon.com/lookout-for-equipment/latest/ug/API_CreateInferenceScheduler.html) API:

In [ ]:
create_scheduler_response = scheduler.create()

Our scheduler is now running and its inference history is currently empty:

![Scheduler created](assets/schedule_created.png)

## Get inference results
---

### List inference executions

**Let's now wait for 5-15 minutes to give some time to the scheduler to run its first inferences.** Once the wait is over, we can use the ListInferenceExecution API for our current inference scheduler. The only mandatory parameter is the scheduler name.

You can also choose a time period for which you want to query inference executions for. If you don't specify it, then all executions for an inference scheduler will be listed. If you want to specify the time range, you can do this:

```python
START_TIME_FOR_INFERENCE_EXECUTIONS = datetime.datetime(2010,1,3,0,0,0)
END_TIME_FOR_INFERENCE_EXECUTIONS = datetime.datetime(2010,1,5,0,0,0)
```

Which means the executions after `2010-01-03 00:00:00` and before `2010-01-05 00:00:00` will be listed.

You can also choose to query for executions in particular status, the allowed status are `IN_PROGRESS`, `SUCCESS` and `FAILED`.

The following cell use `scheduler.list_inference_executions()` as a wrapper around the [**ListInferenceExecutions**](https://docs.aws.amazon.com/lookout-for-equipment/latest/ug/API_ListInferenceExecutions.html) API:

```python
    list_executions_response = lookout_client.list_inference_executions({
        "MaxResults": 50,
        "InferenceSchedulerName": INFERENCE_SCHEDULER_NAME,
        "Status": EXECUTION_STATUS,
        "DataStartTimeAfter": START_TIME_FOR_INFERENCE_EXECUTIONS,
        "DataEndTimeBefore": END_TIME_FOR_INFERENCE_EXECUTIONS
    })
```

In [ ]:
START_TIME_FOR_INFERENCE_EXECUTIONS = None
END_TIME_FOR_INFERENCE_EXECUTIONS = None
EXECUTION_STATUS = None

execution_summaries = []

while len(execution_summaries) == 0:
    execution_summaries = scheduler.list_inference_executions(
        start_time=START_TIME_FOR_INFERENCE_EXECUTIONS,
        end_time=END_TIME_FOR_INFERENCE_EXECUTIONS,
        execution_status=EXECUTION_STATUS
    )
    if len(execution_summaries) == 0:
        print('WAITING FOR THE FIRST INFERENCE EXECUTION')
        time.sleep(60)
        
    else:
        print('FIRST INFERENCE EXECUTED\n')
        break
            
print(len(execution_summaries), 'inference execution(s) found.')
print('Displaying the first three ones:')
execution_summaries[:3]

We have configured this scheduler to run every five minutes. After at least 5 minutes we can also see the history in the console populated with its first few executions: after an hour or so, we will see that the last ones failed as we only generated 12 files above:

![Inference history](assets/schedule_inference_history.png)

When the scheduler starts (for example at `datetime.datetime(2021, 1, 27, 9, 15)`, it looks for **a single** CSV file located in the input location with a filename that contains a timestamp set to the previous step. For example, a file named:

* centrifugal-pump_2021012709**10**00.csv will be found and ingested
* centrifugal-pump_2021012708**15**00.csv will **not be** ingested (it will be ingested at the next inference execution however)

In addition, when opening the file `centrifugal-pump_20210127091000.csv`, it will also open one file before and after this execution time: it will then look for any row with a date that is between the `DataStartTime` and the `DataEndTime` of the inference execution. If it doesn't find such a row in any of these three files, an exception will be thrown: the status of the execution will be marked `Failed`.

### Download inference results
Let's have a look at the content now available in the scheduler output location: each inference execution creates a subfolder in the output directory. The subfolder name is the timestamp (GMT) at which the inference was executed and it contains a single [JSON lines](https://jsonlines.org/) file named `results.jsonl`:

![Inference input](assets/schedule_inference_output_data.png)

Each execution summary is a JSON document that has the following format:

In [ ]:
execution_summaries[0]

When the `Status` key from the previous JSON result is set to `SUCCESS`, you can collect the results location in the `CustomerResultObject` field. We are now going to loop through each execution result and download each JSON lines files generated by the scheduler. Then we will insert their results into an overall dataframe for further analysis:

In [ ]:
results_df = scheduler.get_predictions()
results_df.head()

The content of each JSON lines file follows this format:
    
```json
[
    {
        'timestamp': '2022-03-30T10:46:00.000000',
        'prediction': 1,
        'prediction_reason': 'ANOMALY_DETECTED',
        'anomaly_score': 0.91945,
        'diagnostics': [
            {'name': 'centrifugal-pump\\Sensor0', 'value': 0.12},
            {'name': 'centrifugal-pump\\Sensor1', 'value': 0.0},
            {'name': 'centrifugal-pump\\Sensor2', 'value': 0.0},
                                  .
                                  .
                                  .
            {'name': 'centrifugal-pump\\Sensor27', 'value': 0.08},
            {'name': 'centrifugal-pump\\Sensor28', 'value': 0.02},
            {'name': 'centrifugal-pump\\Sensor29', 'value': 0.02}
        ]
    }
    ...
]
```
Each timestamp found in the file is associated to a `prediction`: 1 when an anomaly is detected an 0 otherwise. You will also have access tothe raw `anomaly_score` ranging from 0.0 to 1.0. This score is normalized so that any raw anomaly score greater than 50% will be associated to a detected anomaly (`prediction = 1`)

When the `prediction` field is 1 (an anomaly is detected), the `diagnostics` field contains each sensor (with the format `component`\\`tag`) and an associated percentage. This percentage corresponds to the magnitude of impact of a given sensor to the detected anomaly. For instance, in the example above, the tag `Sensor0` located on the `centrifugal-pump` component has an estimated 12% magnitude of impact to the anomaly detected at 8pm on April 7th 2021. This dataset has 23 sensors: if each sensor contributed the same way to this event, the impact of each of them would be `100 / 23 = 4.35%`, so 12% is indeed statistically significant.

### Visualizing the inference results
#### Single inference analysis

Each detected event have some detailed diagnostics. Let's unpack the details for the first event and plot a similar bar chart than what the console provides when it evaluates a trained model:

In [ ]:
event_details = pd.DataFrame(results_df.iloc[0, 2:]).reset_index()
fig, ax = lookout.plot.plot_event_barh(event_details)

As we did in the previous notebook, the above bar chart is already of great help to pinpoint what might be going wrong with your asset. Let's load the initial tags description file we prepared in the first notebook and match the sensors with our initial components to group sensors by component:

In [ ]:
# Agregate event diagnostics at the component level:
tags_description_fname = os.path.join(TMP_DATA, 'tags_description.csv')
tags_description_df = pd.read_csv(tags_description_fname)
event_details[['asset', 'sensor']] = event_details['name'].str.split('\\', expand=True)
component_diagnostics = pd.merge(event_details, tags_description_df, how='inner', left_on='sensor', right_on='Tag')[['name', 'value', 'Component']]
component_diagnostics = component_diagnostics.groupby(by='Component').sum().sort_values(by='value')

# Prepare Y position and values for bar chart:
y_pos = np.arange(component_diagnostics.shape[0])
values = list(component_diagnostics['value'])

# Plot the bar chart:
fig = plt.figure(figsize=(12,5))
ax = plt.subplot(1,1,1)
ax.barh(y_pos, component_diagnostics['value'], align='center')
ax.set_yticks(y_pos)
ax.set_yticklabels(list(component_diagnostics.index))
ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))

# Add the values in each bar:
for i, v in enumerate(values):
    ax.text(0.005, i, f'{v*100:.2f}%', color='#FFFFFF', fontweight='bold', verticalalignment='center')
    
# Show the final plot:
plt.show()

#### Multiple inferences analysis

We can also plot the contribution evolution for the top contributing signals over a period of time: the following graph gives an example of what a real time dashboard could look like to expose the results of an Amazon Lookout for Equipment scheduler.

**Note:** The plot stops after a while as there are no more anomaly detected.

In [ ]:
fig = plt.figure(figsize=(24,9))
gs = GridSpec(nrows=3, ncols=1, height_ratios=[1.0, 0.2, 1.0], hspace=0.35)
df = results_df.loc[:, :].copy()

plot_start = np.min(df.index)
plot_end = np.max(df.index)

df = df.loc[plot_start:plot_end]

ax1 = fig.add_subplot(gs[0])
anomaly_plot = ax1.plot(results_df['anomaly_score'], marker='o', markersize=10)
anomaly_plot[0].set_markerfacecolor(colors[5] + '80')
ax1.set_xlim((plot_start, plot_end))
ax1.set_title(f'Centrifugal pump - Live anomaly detection')
ax1.set_xlabel('Raw anomaly score', fontsize=12)

ax3 = fig.add_subplot(gs[1])
lookout.plot.plot_range(results_df, 'Detected events', colors[5], ax3, column_name='prediction')
ax3.set_xlim((plot_start, plot_end))

bar_width = 0.0005
ax4 = fig.add_subplot(gs[2])
bottom_values = np.zeros((len(df.index),))
current_tags_list = list(df.iloc[:, 2:].sum().sort_values(ascending=False).head(8).index)
for tag in current_tags_list:
    plt.bar(x=df.index, height=df[tag], bottom=bottom_values, alpha=0.8, width=bar_width, label=tag.split('\\')[1])
    bottom_values += df[tag].values
    
all_other_tags = [t for t in df.columns if t not in current_tags_list][2:]
all_other_tags_contrib = df[all_other_tags].sum(axis='columns')
plt.bar(x=df.index, height=all_other_tags_contrib, bottom=bottom_values, alpha=0.8, width=bar_width, label='All the others', color='#CCCCCC')

ax4.legend(loc='lower center', ncol=5, bbox_to_anchor=(0.5, -0.45))
ax4.set_xlabel('Signal importance evolution', fontsize=12)
ax4.set_xlim((plot_start, plot_end))

plt.show()

From top to bottom, you can see:
* The evolution of the raw evolution score (real value ranging from 0% to 100%)
* The detected events (binary value, 0 or 1)
* The evolution of the variable importance for each sensor over time

## Inference scheduler operations
---
### Stop inference scheduler
**Be frugal**, running the scheduler is the main cost driver of Amazon Lookout for Equipment. Use the [**StopInferenceScheduler**](https://docs.aws.amazon.com/lookout-for-equipment/latest/ug/API_StopInferenceScheduler.html) API to stop an already running inference scheduler:

```python
stop_scheduler_response = lookout_client.stop_inference_scheduler(
    InferenceSchedulerName=INFERENCE_SCHEDULER_NAME
)
```

The following method is a wrapper around this API call and will stop the periodic inference executions:

In [ ]:
scheduler.stop()

### Start an inference scheduler
You can restart any `STOPPED` inference scheduler using the [**StartInferenceScheduler**](https://docs.aws.amazon.com/lookout-for-equipment/latest/ug/API_StartInferenceScheduler.html) API:

```python
start_scheduler_response = lookout_client.start_inference_scheduler(
    InferenceSchedulerName=INFERENCE_SCHEDULER_NAME
)
```

The following method is a wrapper around this API call and will start the periodic inference executions:

In [ ]:
scheduler.start()

### Delete an inference scheduler
You can delete a **stopped** scheduler you have no more use of: note that you can only have one scheduler per model.

```python
delete_scheduler_response = lookout_client.delete_inference_scheduler(
    InferenceSchedulerName=INFERENCE_SCHEDULER_NAME
)
```

The `scheduler.delete()` method is a wrapper around the [**DeleteInferenceScheduler**](https://docs.aws.amazon.com/lookout-for-equipment/latest/ug/API_DeleteInferenceScheduler.html) API and will delete a stopped scheduler:

In [ ]:
scheduler.stop()
scheduler.delete()

## Conclusion
---

In this notebook, we used the model created in part 3 of this notebook, configured a scheduler and extracted the predictions obtained after it executed a few inferences.

We also showed how we could post-process the inference results to deliver better insights into the detected events.